In [ ]:
# set cwd and solver
import os
import cobra
cobra.Configuration().solver = "gurobi"
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') 

In [ ]:
# load all models in \cs_mods\July2023 folder by cycling through names
from cobra.io import read_sbml_model    
from pathlib import Path
import pandas as pd

# do not change directory but still go to the folder with the models
folder = Path().cwd() / "cs_mods" / "July2023"
model_list = []
for file in folder.glob("*.xml"):  
    model = read_sbml_model(file)
    model.id = file.stem # remove .xml 
    model.name = file.stem # remove .xml
    model_list.append(model)

In [ ]:
# load old combined RPE_PR models
mod_RPE_PR = read_sbml_model(Path().cwd() / "models" / "mod_RPE_PR.xml")
mod_RPE_PR.id = 'RPE_PR_old'
mod_RPE_PR.name = 'RPE_PR_old'
mod_Human1_Human1 = read_sbml_model(Path().cwd() / "models" /  "mod_Human1_Human1.xml")
mod_Human1_Human1.id = 'RPE_PR_control_old'
mod_Human1_Human1.name = 'RPE_PR_control_old'
mod = read_sbml_model(Path().cwd() / 'models/Human-GEM.xml')
mod.id = 'Human1_old'
mod.name = 'Human1_old'

# add old models to model_list
model_list = model_list + [mod,mod_RPE_PR]

# check out model_list
model_list

In [ ]:
# modify models
# load blood exchange bounds

# for all models in the list set objective function (ATP hydrolysis, in PR for combined models)
for m in model_list:
    if 'MAR03964_PR' in [r.id for r in m.reactions]:
        m.objective = 'MAR03964_PR'
    elif 'MAR03964_RPE' in [r.id for r in m.reactions]:
        m.objective = 'MAR03964_RPE'
    elif 'MAR03964' in [r.id for r in m.reactions]:
        m.objective = 'MAR03964'
    else:
        print('no ATP hydrolysis reaction (MAR03964) in model: ' + m.id)

# list combined and single models
combined_models = [m for m in model_list if 'single' not in m.id and 'Human' not in m.id]
single_models = [m for m in model_list if 'single' in m.id]
Human1_models = [m for m in model_list if 'Human' in m.id]

# close PR exchange in combined models and open RPE exchange (efflux)
for m in combined_models:
    for r in [r for r in m.reactions if len(r.products) == 0 ]:           # close all exchange reactions
        r.bounds=(0,0)
    for r in [r for r in m.reactions if len(r.products) == 0 if '_RPE' in r.id]:           # open efflux for RPE exchange reactions
        r.bounds=(0,1000) 

# only allow efflux exchange in single models
for m in single_models:
    for r in [r for r in m.reactions if len(r.products) == 0 ]:           # close all exchange reactions
        r.bounds=(0,1000)

# for Human1 models, open efflux
for m in [m for m in model_list if 'Human' in m.id]:
    for r in [r for r in m.reactions if len(r.products) == 0]:           # open efflux for RPE exchange reactions
        r.bounds=(0,1000)

In [ ]:
# set exchange bounds for blood
from src.modify_model import set_exchange_bounds
ex_bounds = {'MAR09048':(-3.51,1000),'MAR09034':(-4.18,1000),'MAR09135':(7.25,1000)}
# MAR09048: oxygen
# MAR09034: glucose
# MAR09135: lactate

results_dict = {}
for m in model_list:
    set_exchange_bounds(m, ex_bounds)
    m.optimize()
    results_dict[m.id] = m.objective.value
df1 = pd.DataFrame(results_dict, index = ['max ATP hydrolysis (pmol/s/mm^2)']).T
df1

In [ ]:
model_list[-5]

In [ ]:
# This cell is disabled because it takes a long time to run.
# The results are saved in the folder 'results/unlimited_lactate' and can be loaded from there.
# To enable this cell, remove the comment symbol (#) at the beginning of each line

# FLUX VARIABILITY ANALYSIS

# ex_bounds = [{'MAR09048':(-3.51,1000),'MAR09034':(-4.18,1000),'MAR09135':(-1000,1000)},\
#             {'MAR09048':(0,1000),'MAR09034':(-4.18,1000),'MAR09135':(-1000,1000)},\
#             {'MAR09048':(-1000,1000),'MAR09034':(-4.18,1000),'MAR09135':(-1000,1000)}]

# MAR09048: oxygen
# MAR09034: glucose
# MAR09135: lactate

# from cobra.flux_analysis import flux_variability_analysis

# fba = {}
# fva = {}
# fva_loopless = {}

# for bounds in ex_bounds:
#     m = model_list[-5]
#     set_exchange_bounds(m, bounds)
#     fba[str(bounds)] = pd.DataFrame(m.optimize().fluxes)
#     print('fba done with bounds: ' + str(bounds) + ' for model: ' + m.id)
#     fva[str(bounds)] = flux_variability_analysis(m, loopless=False)
#     print('fva done with bounds: ' + str(bounds) + ' for model: ' + m.id)
#     fva_loopless[str(bounds)] = flux_variability_analysis(m, loopless=True)
#     print('fva_loopless done with bounds: ' + str(bounds) + ' for model: ' + m.id)

# # pickle fba, fva and fva_loopless
# from pathlib import Path
# # create folder if it does not exist
# Path(Path().cwd() / 'results' / 'unlimited_lactate').mkdir(parents=True, exist_ok=True)

# import pickle
# with open(Path().cwd() / 'results' / 'unlimited_lactate' / 'fba.pkl', 'wb') as f:
#     pickle.dump(fba, f)
# with open(Path().cwd() / 'results' / 'unlimited_lactate' / 'fva.pkl', 'wb') as f:
#     pickle.dump(fva, f)
# with open(Path().cwd() / 'results' / 'unlimited_lactate' / 'fva_loopless.pkl', 'wb') as f:
#     pickle.dump(fva_loopless, f)

In [ ]:
# Load FBA and FVA results for unlimited lactate production (output from cell above)

import pickle
with open(Path().cwd() / 'results' / 'unlimited_lactate' / 'fba.pkl', 'rb') as f:
    fba = pickle.load(f)
with open(Path().cwd() / 'results' / 'unlimited_lactate' / 'fva.pkl', 'rb') as f:
    fva = pickle.load(f)
with open(Path().cwd() / 'results' / 'unlimited_lactate' / 'fva_loopless.pkl', 'rb') as f:
    fva_loopless = pickle.load(f)   

In [ ]:
# fba
df_fba = pd.concat(fba, axis=1)
df_fba = df_fba.rename(columns={'fluxes': 'fba'})

# fva
df_fva = pd.concat(fva, axis=1)
df_fva = df_fva.rename(columns={'minimum': 'min', 'maximum': 'max'})

# loopless fva
df_fva_loopless = pd.concat(fva_loopless, axis=1)
df_fva_loopless = df_fva_loopless.rename(columns={'minimum': 'min_loopless', 'maximum': 'max_loopless'})

# make one big df with fba, fva, loopless fva
df = pd.concat([df_fba, df_fva, df_fva_loopless], axis=1)

# resort columns based on first layer of multiindex per first multi index level
df = df.reindex(sorted(df.columns), axis=1)

# for first layer multiindex
for i in range(len(df.columns.levels[0])):
    # sort second layer multiindex ['fba', 'min', 'max', 'min_loopless', 'max_loopless']
    df = df.reindex(['fba', 'min', 'min_loopless','max','max_loopless'], axis=1, level=1)


In [ ]:
# sort rows based on absolute value sizes in min_loopless and max_loopless columns
# get absolute values for loopless columns
# select columns with 'min_loopless' or 'max_loopless' in the second level of the column index
min_max_loopless = df.loc[:, df.columns.get_level_values(1).isin(['min_loopless', 'max_loopless'])]
# transform to absolute values
min_max_loopless = min_max_loopless.abs()
# get sum of absolute values for each row
min_max_loopless['sum'] = min_max_loopless.sum(axis=1)

In [ ]:
# make rxn_df
from src.get_info import make_rxn_df
rxn_df = make_rxn_df(model_list[-5])

In [ ]:
# fuse rxn_df and df keep rows sorted based on df 
df_combined = pd.concat([rxn_df, df], axis=1)
# sort df using min_max_loopless['sum']
df_combined = df_combined.reindex(min_max_loopless.sort_values('sum', ascending=False).index)

# save df_combined as excel in results folder / fva_lactate_unlimited
df_combined.to_excel('results/unlimited_lactate/unlimited_lactate.xlsx')
df_combined

In [ ]:
# select oxygen and lactate exchange reactions, and PR ATP hydrolysis reaction

# 'MAR03964_PR' PR ATP hydrolysis
# 'MAR09048_RPE' oxygen
# 'MAR09135_RPE' lactate

df_combined_selection = df_combined.loc[['MAR03964_PR', 'MAR09048_RPE', 'MAR09135_RPE'], :]
df_combined_selection = df_combined.loc[['MAR03964_PR', 'MAR09135_RPE'], :]
df_combined_selection


In [ ]:
# select min loopless columns only (columns containing 'min_loopless')
df_min_loopless = df_combined_selection[[c for c in df_combined_selection.columns if 'min_loopless' in c]]
df_max_loopless = df_combined_selection[[c for c in df_combined_selection.columns if 'max_loopless' in c]]

df_max_loopless.transpose()

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

# plot
fig, ax = plt.subplots(figsize=(6, 4))
df_min_loopless.transpose().plot(ax=ax, legend=False)
ax.set_ylabel('Flux (pmol/s/mm2)')
for item in ax.get_xticklabels():
    item.set_rotation(0)
ax.set_xticks([0,1,2])
ax.set_xticklabels(['unlimited oxygen','limited oxygen (3.51)','no oxygen'])
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5)) 
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, ['ATP', 'lactate'], loc='center left', bbox_to_anchor=(1, 0.5))


plt.title('Minimum fluxes (loopless)')
fig, ax = plt.subplots(figsize=(6, 4))
df_max_loopless.transpose().plot(ax=ax, legend=False)
ax.set_ylabel('Flux (pmol/s/mm2)')
for item in ax.get_xticklabels():
    item.set_rotation(0)
ax.set_xticks([0,1,2])
ax.set_xticklabels(['unlimited oxygen','limited oxygen (3.51)','no oxygen'])
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, ['ATP', 'lactate'], loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('Maximum fluxes (loopless)')


In [ ]:
# plot minimum and maximum lines
df_min_max_loopless = df_combined_selection[[c for c in df_combined_selection.columns if 'max_loopless' in c or 'min_loopless' in c]]

df_min_max_loopless_ATP = df_min_max_loopless.filter(like='MAR03964_PR', axis=0) # select MAR03964_PR dat
df_min_max_loopless_lactate = df_min_max_loopless.filter(like='MAR09135_RPE', axis=0)# select MAR09135_RPE data

In [ ]:
# select min and max loopless columns only (columns containing 'min_loopless' or 'max_loopless')
df_min_max_loopless = df_combined_selection[[c for c in df_combined_selection.columns if 'min_loopless' in c or 'max_loopless' in c]]

# create plot
fig, ax = plt.subplots(figsize=(6, 4))

# plot minimum and maximum lines

# select MAR03964_PR data (ATP)
df_min_max_loopless_ATP = df_min_max_loopless.filter(like='MAR03964_PR', axis=0)
# select MAR09135_RPE data (lactate)
df_min_max_loopless_lactate = df_min_max_loopless.filter(like='MAR09135_RPE', axis=0)

df_min_max_loopless_ATP.filter(like='min_loopless').transpose().plot(ax=ax, legend=False, color='r')
df_min_max_loopless_ATP.filter(like='max_loopless').transpose().plot(ax=ax, legend=False, color='r')
df_min_max_loopless_lactate.filter(like='min_loopless').transpose().plot(ax=ax, legend=False, color='b')
df_min_max_loopless_lactate.filter(like='max_loopless').transpose().plot(ax=ax, legend=False, color='b')

ax.set_ylabel('Flux (pmol/s/mm2)')
for item in ax.get_xticklabels():
    item.set_rotation(0)
ax.set_xticks([0,1,2])
ax.set_xticklabels(['unlimited oxygen','limited oxygen (3.51)','no oxygen'])
ax.legend(['ATP min', 'ATP max', 'lactate min', 'lactate max'], loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('Minimum and maximum fluxes (loopless)')
plt.show()